<a href="https://colab.research.google.com/github/arora123/Python-for-Data-Science/blob/master/DataFrameMapper_Vs_Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/scikit-learn-contrib/sklearn-pandas/issues/173#issuecomment-689606176

DataFrameMapper() from sklearn_pandas and ColumnTransformer from sklearn.compose

both helps to apply different transformations [imputers, scalars, encoders, polynomial terms, etc.] on different columns

#Importing libraries and Getting Data

In [9]:
import pandas as pd

#For Polynomial Features
from sklearn.preprocessing import PolynomialFeatures

#DataMapper for applying transformations on selected columns
from sklearn_pandas import DataFrameMapper

#ColumnTransformer for applying transformations on selected columns
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector 
from sklearn.pipeline import Pipeline # for pielines

# Loading Data
housing = pd.read_csv('https://raw.githubusercontent.com/arora123/Data/master/housing.csv')

# Creating interaction term [optional]
housing['houseAge_income'] = housing['housing_median_age']*housing['median_income']

In [5]:
degree = 2

mapper = DataFrameMapper([
(['houseAge_income'], PolynomialFeatures(degree=degree, include_bias=False)),
(['median_income'], PolynomialFeatures(degree=degree, include_bias=False)),
])

poly_feature = mapper.fit_transform(housing)

# print(poly_feature)
# print(mapper.transformed_names_)

poly_dm = pd.DataFrame(poly_feature, columns=mapper.transformed_names_)
poly_dm.head()

,houseAge_income_x0,houseAge_income_x0^2,median_income_x0,median_income_x0^2
0,341.3332,116508.353422,8.3252,69.308955
1,174.3294,30390.739704,8.3014,68.913242
2,377.3848,142419.287271,7.2574,52.669855
3,293.4412,86107.737857,5.6431,31.844578
4,200.0024,40000.960006,3.8462,14.793254


In [6]:
housing[['houseAge_income', 'median_income']].head()

,houseAge_income,median_income
0,341.3332,8.3252
1,174.3294,8.3014
2,377.3848,7.2574
3,293.4412,5.6431
4,200.0024,3.8462


In [7]:
poly_dm.describe()

,houseAge_income_x0,houseAge_income_x0^2,median_income_x0,median_income_x0^2
count,20640.000000,20640.000000,20640.000000,20640.000000
mean,108.008027,17198.805116,3.870671,18.591242
std,74.386418,33467.667120,1.899822,21.946544
min,1.625000,2.640625,0.499900,0.249900
25%,57.997425,3363.701325,2.563400,6.571020
50%,92.502950,8556.795767,3.534800,12.494811
75%,140.104900,19629.383139,4.743250,22.498421
max,780.005200,608408.112027,15.000100,225.003000


In [8]:
housing[['houseAge_income', 'median_income']].describe()

,houseAge_income,median_income
count,20640.000000,20640.000000
mean,108.008027,3.870671
std,74.386418,1.899822
min,1.625000,0.499900
25%,57.997425,2.563400
50%,92.502950,3.534800
75%,140.104900,4.743250
max,780.005200,15.000100


In [28]:
Poly_features = ['houseAge_income', 'median_income']

poly_transformer = Pipeline(steps=[
    ('poly features', PolynomialFeatures(degree, include_bias=False)),
    # ('poly features', PolynomialFeatures(degree, include_bias=False, interaction_only=True))
    ])

pipe1 = ColumnTransformer(transformers=[
    ('poly', poly_transformer, Poly_features)])

In [29]:
poly_ct = pd.DataFrame(pipe1.fit_transform(housing))

In [30]:
poly_ct.head() # Columns for 'houseAge_income', 'median_income', 'houseAge_income^2', 'median_income^2' and 'houseAge_income'*'median_income'

# If we use interaction_only=True inside PolynomialFeatures(), we will get 'houseAge_income', 'median_income' and 'houseAge_income'*'median_income' columns


,0,1,2,3,4
0,341.3332,8.3252,116508.353422,2841.667157,69.308955
1,174.3294,8.3014,30390.739704,1447.178081,68.913242
2,377.3848,7.2574,142419.287271,2738.832448,52.669855
3,293.4412,5.6431,86107.737857,1655.918036,31.844578
4,200.0024,3.8462,40000.960006,769.249231,14.793254


In [21]:
pipe1.get_feature_names

<bound method ColumnTransformer.get_feature_names of ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('poly',
                                 Pipeline(memory=None,
                                          steps=[('poly features',
                                                  PolynomialFeatures(degree=2,
                                                                     include_bias=True,
                                                                     interaction_only=False,
                                                                     order='C'))],
                                          verbose=False),
                                 ['houseAge_income', 'median_income'])],
                  verbose=False)>